In [1]:
import pandas as pd
import sys,os
from operator import itemgetter
sys.path.insert(0, '/mnt/c/users/cojoe/Python-Stuff')
import JCLib
import threading
import PySimpleGUI as sg
import pyglet,tkinter
from pyglet import font
# import OpenGL
# from OpenGL import GLU
font.add_file('/etc/fonts/fonts/CENTAUR.TTF')

##  Variables related to system info or imports
platform = sys.platform 


In [13]:
def getFile(how,file,WindowP):
    if how == "Local":
        if file[-3:].lower() == "tsv":
            delim = "\t"
            df=pd.read_csv(file,encoding="latin",delimiter=delim)
        elif file[-4:].lower() == "xlsx":
            df=pd.read_excel(file,engine="openpyxl")
        else:
            try:
                df=pd.read_csv(file)
            except Exception as err:
                print("Error, trying with encoding=latin")
                df=pd.read_csv(file,encoding="latin")
    elif how == "Fetch":
        print("Getting that web data")
        file=values["-WEB-"]
        getPrevFiles(2,file)

        windowP["-PINFO-"].update(f"START reading WEB File:{file}:")
        df=pd.read_csv(file)
        windowP["-PINFO-"].update(f"FINISHED reading WEB File:{file}:")
        
    return df

##############################################################
def setRowColors(lst,col1,col2,colsp,header):
    count=0
    colors = {}
    nrec = header.index("% Missing")
    for vals in lst:
        key = vals[0]
        if count%2 == 0:
            colors[key] = col1
        else:
            colors[key] = col2
        if vals[nrec] > 99.0:
            colors[key] = colsp
        count+=1
    colTab = []
    for key,colr in colors.items():
        colTab.append(colr)
    rowNums = [num for num in range(0,len(colTab)+1)]
    colText = ["black"]*len(colTab)
    colrw = list(zip(rowNums,colTab))
        
    return colrw
############################################################

def getRowClicked(table):
        
    e = table.user_bind_event 
    region = table.Widget.identify('region', e.x, e.y)
    if region == 'heading':
        row = 0
    elif region == 'cell':
        row = int(table.Widget.identify_row(e.y))
        
    return row


###########################################################
def sortTable(row,stats,colSortState,table,event,header):
    
#         e = table.user_bind_event 
#         region = table.Widget.identify('region', e.x, e.y)
#         if region == 'heading':
#             row = 0
#         elif region == 'cell':
#             row = int(table.Widget.identify_row(e.y))
   
        if row == 0:
            colClicked = int(table.Widget.identify_column(e.x)[1:])
            statClicked = header[colClicked-1].strip()
            colSortState[statClicked]*=-1
            if colSortState[statClicked] == -1:
                sortAsc=False
            else:
                sortAsc=True
            if colClicked > 1:  # user number sort
                statsS = dict(sorted(stats.items(), key=lambda x: x[1][statClicked],reverse=sortAsc))
            else:
                statsS = dict(sorted(stats.items(), key=lambda x: x[0],reverse=sortAsc))


            statsVals=[]
            for col in statsS:
                 vals=[]
                 vals.append(col)
                 for k in header[1:]:
                    vals.append(statsS[col][k.strip()])
                 statsVals.append(vals)
           # slen= len(statsVals)
#             colorsTable = setRowColors(statsVals,"#b3f0ff","#33d6ff","pink",header_list)

#             window['-TABLE-'].update(values=statsVals,row_colors=colorsTable)
        return statsVals,colSortState


In [14]:
def dfAnalyze(df):
    stats = {}
    
    for col in df.columns:
        typs = df[col].apply(type).value_counts().to_dict()
        stats[col] = {}
        if str in typs:
           stats[col]["string"] = typs[str]
        else:
           stats[col]["string"] = 0

        if int in typs:
           stats[col]["integer"] = typs[int]
        else:
           stats[col]["integer"] = 0

        if float in typs:
           stats[col]["float"] = typs[float]
        else:
           stats[col]["float"] = 0

        if bool in typs:
           stats[col]["boolean"] = typs[bool]
        else:
           stats[col]["boolean"] = 0

        stats[col]["Missing"] = df[col].isna().sum()
        stats[col]["% Missing"] = round(df[col].isna().sum()/df.shape[0]*100,1)
        
    return stats
#############################################  
def getValues(stats,header):

    statsVals=[]
    for col in sorted(stats.keys()):
         vals=[]
         vals.append(col)
         for k in header[1:]:
            vals.append(stats[col][k.strip()])
         statsVals.append(vals)
    return statsVals
#############################################
def showCompare(stats1,stats2,df1,df2,file1,file2):
    header_list = ["Column","% Missing","Missing","string","integer","float","boolean"]
    
    col_widths = [8]*len(header_list)
    col_widths[0] = 25
    columnSortStateTable1 = {}
    columnSortStateTable2 = {}

    ## set up sort state for the column in both tables
    for col in header_list:
         columnSortStateTable1[col] = -1
         columnSortStateTable2[col] = -1
            
    valsFile1 = getValues(stats1,header_list)
    valsFile2 = getValues(stats2,header_list)
    
    rowFile1Colors = setRowColors(valsFile1,"#b3f0ff","#33d6ff","pink",header_list)
    rowFile2Colors = setRowColors(valsFile2,"#8AF5A1","#2DD150","pink",header_list)
    
    
    
    layCol1 = [[sg.Text(f"File 1 {file1}",font="CENTAUR 15")],[sg.Text(f"File 1 Shape {df1.shape}",font="CENTAUR 15")],
               [sg.Table(values=valsFile1,text_color="black", auto_size_columns=False,enable_events=True,num_rows=20,col_widths=col_widths,font="CENTAUR 10",
                   justification='center',pad=(5,5),vertical_scroll_only=False,
                   key='-TABLEFILE1-',row_colors=rowFile1Colors,headings = header_list)]
               ]
    
    layCol2 = [[sg.Text(f"File 2 {file2}",font="CENTAUR 15")],[sg.Text(f"File 2 Shape {df2.shape}",font="CENTAUR 15")],
               [sg.Table(values=valsFile2,text_color="black", auto_size_columns=False,enable_events=True,num_rows=20,col_widths=col_widths,font="CENTAUR 10",
                   justification='center',pad=(5,5),vertical_scroll_only=False,
                   key='-TABLEFILE2-',row_colors=rowFile2Colors,headings = header_list)]      
              ]
    
    layout = [[sg.Button("Quit")],[sg.Button("Compare 2 Columns")],[
     sg.Column(layCol1),sg.Column(layCol2) ,]  
        
    ]
#     layout = [[sg.Button("Quit")],
#              [sg.Text(f"File 1 {file1}",font="CENTAUR 10"),sg.Text(f"File 2 {file2}",font="CENTAUR 10")],
              
#              [sg.Text(f"File 1 {file1}",font="CENTAUR 10"),sg.Text(f"File 2 Shape {df2.shape}",font="CENTAUR 10")],
        
#                [sg.Table(values=valsFile1,text_color="black", auto_size_columns=False,enable_events=True,num_rows=20,col_widths=col_widths,font="CENTAUR 10",
#                    justification='center',pad=(5,5),vertical_scroll_only=False,
#                    key='-TABLEFILE1-',row_colors=rowFile1Colors,headings = header_list),

#                sg.Table(values=valsFile2,text_color="black", auto_size_columns=False,enable_events=True,num_rows=20,col_widths=col_widths,font="CENTAUR 10",
#                    justification='center',pad=(5,5),vertical_scroll_only=False,
#                    key='-TABLEFILE2-',row_colors=rowFile2Colors,headings = header_list)],
#              ]
             
              
              
    window2 = sg.Window('Files',layout,finalize=True,resizable=True)
    a = window2.CurrentLocation()
    screen_width, screen_height = window2.get_screen_dimensions()
    win_width, win_height = window2.size
    x, y = (screen_width - win_width)//2, (screen_height - win_height)//2
    x=200
    y=200
    window2.move(x, y)
    tableFile1 = window2['-TABLEFILE1-']
    tableFile1.bind('<Button-1>', "Click")
    tableFile2 = window2['-TABLEFILE2-']
    tableFile2.bind('<Button-1>', "Click")
    
    while True:
        try:
           
            event, values = window2.read(timeout=100000)
            print(event)
            print(values)
            if event == sg.WIN_CLOSED or event == 'Quit':
                window2.close()
                break
            elif event == "-TABLEFILE1-Click":
                print("Table 1 clicked")
                row=getRowClicked(tableFIle1)
                if row == 0:
                   valsFile1,columnSortStateTable1 = sortTable(stats1,columnSortStateTable1,tableFile1,event,header_list)          
                   setRowColors(valsFile1,"#b3f0ff","#33d6ff","pink",header_list)
                   window2["-TABLEFILE1-"].update(values=valsFile1,row_colors=rowFile1Colors)
            elif event == "-TABLEFILE2-Click":
                print("Table 2 clicked")
                row=getRowClicked(tableFIle2)
                if row == 0:
                    valsFile2,columnSortStateTable2 = sortTable(stats2,columnSortStateTable2,tableFile2,event,header_list)
                    rowFile2Colors = setRowColors(valsFile2,"#8AF5A1","#2DD150","pink",header_list)
                    window2["-TABLEFILE2-"].update(values=valsFile2,row_colors=rowFile2Colors)
            elif event == "Compare 2 Columns":
                if len(values["-TABLEFILE1"]) > 0:
                    colF1 = values["-TABLEFILE1"]
                    print(valsFile1)
                if len(values["-TABLEFILE2"]) > 0:
                    colF2 = values["-TABLEFILE2"]
                
                print("OK")
        except Exception as err:
            print("Error in WIndow 2: ",err)
    
##############################################    
def fileBrowser():
    global df1,df2
    global stats1,stats2

    # previousFiles = getPrevFiles(1)
    # previousFiles = list(previousFiles.keys())
    # print("Create Layout")
    layout = [[sg.Button("Quit")],
             [sg.FilesBrowse(button_text="File # 1",initial_folder="/home/joe/bic_etl",font="CENTAUR 15",file_types=[("CSV Files","*.csv"),("TSV Files","*.tsv"),("Excel Files","*.xlsx")],enable_events=True,key='-FILE1-')],
             [sg.FilesBrowse(button_text="File # 2",initial_folder="/home/joe/bic_etl",font="CENTAUR 15",file_types=[("CSV Files","*.csv"),("TSV Files","*.tsv"),("Excel Files","*.xlsx")],enable_events=True,key='-FILE2-')],  
             [sg.Text("Web Address:",font="CENTAUR 15"),sg.Input(default_text="",font="CENTAUR 15",key="-WEB1-")],
             [sg.Text("Web Address:",font="CENTAUR 15"),sg.Input(default_text="",font="CENTAUR 15",key="-WEB2-")],             
             [sg.Button("Compare")],
             [sg.Multiline(default_text="Summary\n",key="-PINFO-",font="CENTAUR 10",size=[70,5])],
             ]
              
              
    window = sg.Window('Files',layout,finalize=True,resizable=True)
    
    a = window.CurrentLocation()
    
    
    screen_width, screen_height = window.get_screen_dimensions()
    win_width, win_height = window.size
    x, y = (screen_width - win_width)//2, (screen_height - win_height)//2
 
    x=200
    y=200
    window.move(x, y)

    
    while True:
        try:
           
            event, values = window.read(timeout=100000)
            print(event)
            print(values)
            if event == sg.WIN_CLOSED or event == 'Quit':
                window.close()
                break
            elif event == "Compare":
                files=[]
                values["-FILE1-"] = "/home/joe/bic_etl/cdos/business/nonprofit/data_source/char_orgs_sol.tsv"
                values["-FILE2-"] = "/home/joe/bic_etl/cdos/business/nonprofit/data_transformed/char_orgs_sol.csv"
 
                if len(values["-FILE1-"]) > 0:
                     df1 = getFile("Local",values["-FILE1-"],window)
                     stats1 = dfAnalyze(df1)
                    
                     files.append(values["-FILE1-"])
                if len(values["-FILE2-"]) > 0:
                     df2 = getFile("Local",values["-FILE2-"],window)
                     stats2 = dfAnalyze(df2)
                    

                     files.append(values["-FILE2-"])
                if len(values["-WEB1-"]) > 0:
                     files.append(values["-WEB1-"])
                if len(values["-WEB2-"]) > 0:
                     files.append(values["-WEB2-"])
                file1=values["-FILE1-"]
                file2=values["-FILE2-"]
                
                showCompare(stats1,stats2,df1,df2,file1,file2)
              
            elif event == "-FILE1-":
                window["-PINFO-"].update(f"Local FIle:{values['-FILE1-']}:",append=True)
            elif event == "-FILE2-":
                window["-PINFO-"].update(f"Local FIle:{values['-FILE2-']}:",append=True)
        except Exception as err:
              print("Error: ",err)
                    
fileBrowser()

Compare
{'-FILE1-': '', '-FILE2-': '', '-WEB1-': '', '-WEB2-': '', '-PINFO-': 'Summary'}


/tmp/ipykernel_1868/2540449515.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file,encoding="latin",delimiter=delim)
/tmp/ipykernel_1868/2540449515.py:10: DtypeWarning: Columns (1,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file)


-TABLEFILE1-Click
{'-TABLEFILE1-': [], '-TABLEFILE2-': []}
Table 1 clicked
Error in WIndow 2:  name 'tableFIle1' is not defined
-TABLEFILE1-
{'-TABLEFILE1-': [4], '-TABLEFILE2-': []}
Quit
{'-TABLEFILE1-': [4], '-TABLEFILE2-': []}
Quit
{'-FILE1-': '', '-FILE2-': '', '-WEB1-': '', '-WEB2-': '', '-PINFO-': 'Summary'}


In [ ]:
stats1

In [ ]:
stats2

In [ ]:
df1
